# Benchmarking cities in MENA

In support of an upcoming Urban flagship report, the MENA team is looking for a series of zonal statistics:

- Nighttime Lights, Population, and built-area:  
  - Entire FUA  
  - Its associated urban center / “core”  
  - Associated “periphery”  

The unit of analysis is the Functional Urban Areas (FUAs) from the [UCDB Database](https://human-settlement.emergency.copernicus.eu/ghs_stat_ucdb2015mt_r2019a.php). For each FUA, we need to grab the associated urban periphary (lower threshold urban areas)

In [1]:
import sys
import os

import geopandas as gpd
import pandas as pd

sys.path.append("/home/wb411133/Code/GOSTrocks/src")

import GOSTrocks.ntlMisc as ntlMisc
import GOSTrocks.rasterMisc as rMisc

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
data_folder = "s3://wbg-geography01/URBANIZATION/MENA/Extents/"
ucdb_file       = "/home/wb411133/Code/GOSTurban/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg"
fua_file        = os.path.join(data_folder, "GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg")
fua_peripheries = os.path.join(data_folder, "FUA_peripheries.gpkg")

In [12]:
inU = gpd.read_file(ucdb_file)
# If the peripheries exists read them in, if not, create them
inF = gpd.read_file(fua_file)
inF = inF.to_crs(inU.crs)
try:
    inP = gpd.read_file(fua_peripheries)
except:
    
    fua_peripheries = inF.copy()
    for idx, row in inF.iterrows():
        # grab the related UCDBs
        ucdb_ids = row["UC_IDs"].split(";")
        ucdb_ids = [int(x) for x in ucdb_ids]
        sel_cores = inD.loc[inD["ID_HDC_G0"].isin(ucdb_ids)]
        periphery_geom = row["geometry"].difference(sel_cores.unary_union)
        fua_peripheries.loc[idx, "geometry"] = periphery_geom

    fua_peripheries.to_file(os.path.join(out_folder, "FUA_peripheries.gpkg"), driver="GPKG")
    inP = fua_peripheries
inP = inP.to_crs(inU.crs)
inP['geometry'] = inP.buffer(0)

In [13]:
fua_res = "/home/wb411133/temp/fua_ntl_zonal.csv"
try:
    fua_zonal = pd.read_csv(fua_res)
except:
    fua_zonal = ntlMisc.run_zonal(inF, verbose=True)
    fua_zonal.to_csv(fua_res)

08:19:09	20230601
08:21:34	20230701
08:24:18	20230901
08:27:04	20231001
08:29:51	20231101
08:32:39	20231201
08:35:25	20240101
08:38:05	20240201
08:40:46	20240301
08:43:33	20240401
08:46:21	20120119
08:49:03	20120201
08:51:43	20120301
08:54:25	20120401
08:57:04	20120501
08:59:41	20120601
09:02:20	20120701
09:05:04	20120801
09:07:48	20120901
09:10:31	20121001
09:13:18	20121101
09:16:02	20121201
09:18:48	20130101
09:21:32	20130201
09:24:12	20130301
09:26:50	20130401
09:29:29	20130501
09:32:13	20130601
09:34:53	20130701
09:37:29	20130801
09:40:10	20130901
09:42:48	20131001
09:45:26	20131101
09:48:06	20131201
09:50:40	20140101
09:53:22	20140201
09:56:03	20140401
09:58:42	20140501
10:01:23	20140601
10:04:12	20140701
10:07:01	20140801
10:09:52	20140901
10:12:46	20141001
10:15:37	20141101
10:18:35	20141201
10:21:30	20150101
10:24:30	20150201
10:27:21	20150301
10:30:11	20150401
10:33:06	20150501
10:35:56	20150601
10:38:42	20150701
10:41:33	20150801
10:44:24	20150901
10:47:10	20151001
10:50:00	2

In [15]:
core_res = "/home/wb411133/temp/cores_ntl_zonal.csv"
try:
    core_zonal = pd.read_csv(core_res)
except:
    core_zonal = ntlMisc.run_zonal(inU, verbose=True)
    core_zonal.to_csv(core_res)

In [ ]:
''' The clipping process produces multi and null geometries, which breaks this process
#### TODO: Determine if this is necessary
per_res = "/home/wb411133/temp/periphary_ntl_zonal.csv"
try:
    per_zonal = pd.read_csv(per_res)
except:
    per_zonal = ntlMisc.run_zonal(inP, verbose=True)
    per_zonal.to_csv(per_res)
'''

## Combine NTL results
We have zonal results for the entire FUA and for the core as monthly results; there is a two step process to create the final results:
1. Combine monthly results into annual results  
2. Use FUA and cores to generate three stats: FUA SoL, Core SoL, and Periphery SoL

In [33]:
def combine_ntl_annual(curD):
    '''curD is a data frame of ntl zonal results'''
    for yr in range(2012, 2024):
        cur_columns = [x for x in curD.columns if f"ntl_{yr}" in x]
        sel_d = curD.loc[:,cur_columns]
        curD[f'ntl{yr}_SoL'] = sel_d.sum(axis=1)/len(cur_columns)
    return(curD)

out_folder = "s3://wbg-geography01/URBANIZATION/MENA/ZONAL_RES/NTL/"
combine_ntl_annual(fua_zonal).to_csv(os.path.join(out_folder, "fua_VIIRS_SoL.csv"))
combine_ntl_annual(core_zonal).to_csv(os.path.join(out_folder, "core_VIIRS_SoL.csv"))

In [32]:
combine_ntl_annual(core_zonal)

,Unnamed: 0,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,...,ntl2013_SoL,ntl2014_SoL,ntl2015_SoL,ntl2016_SoL,ntl2017_SoL,ntl2018_SoL,ntl2019_SoL,ntl2020_SoL,ntl2021_SoL,ntl2022_SoL
0,0,1.0,1.0,185.0,21.247683,-158.043016,21.422193,-157.730529,21.340678,-157.893497,...,23184.833496,22703.133700,22175.290039,21412.786621,22074.702637,21063.317383,20878.145345,20514.217773,20648.978871,21291.070312
1,1,2.0,2.0,42.0,-17.641184,-149.628088,-17.517631,-149.508018,-17.534103,-149.568053,...,2025.002502,2071.165483,1872.035848,1780.467499,1721.464152,1543.889160,1569.392517,1498.755829,1504.045477,1353.600037
2,2,3.0,1.0,55.0,34.858517,-120.475511,34.989334,-120.389183,34.923123,-120.434372,...,5543.114115,5591.504528,5667.249146,5702.728394,5934.019938,5043.245036,4768.488241,4968.653341,5118.384566,5827.381934
3,3,4.0,1.0,48.0,36.582997,-121.952215,36.635743,-121.811816,36.607720,-121.882378,...,3952.295105,3771.239080,3825.278280,3488.866679,3423.468363,2899.663350,3021.289978,2835.289195,2869.639948,3314.380994
4,4,5.0,1.0,60.0,34.388220,-119.853855,34.457831,-119.658413,34.427664,-119.743693,...,6900.397583,6785.974565,6515.661662,6373.352539,6321.239095,5717.559204,5332.768148,5020.043315,5048.719039,5338.875049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13130,13130,13131.0,1.0,70.0,-37.737746,176.104748,-37.631690,176.321892,-37.684080,176.183663,...,6363.823324,6420.390891,6449.661499,6121.288371,6486.337402,5359.219828,6339.664185,5906.040894,5797.636275,5517.883936
13131,13131,13132.0,0.0,3.0,-6.759222,155.679978,-6.743003,155.703239,-6.749761,155.693009,...,2.912500,2.881818,3.417500,3.090833,3.167500,2.506667,2.606667,7.964167,11.018182,7.797000
13132,13132,13133.0,1.0,23.0,-9.454775,159.911539,-9.422255,160.006750,-9.436571,159.963101,...,386.770833,342.587277,414.412506,497.889994,510.835841,443.670835,522.494998,673.990824,611.004550,489.574011
13133,13133,13134.0,1.0,27.0,-22.304278,166.418428,-22.245962,166.483047,-22.277587,166.452136,...,2640.184153,3059.595459,2709.849162,2795.125000,2786.766663,2454.539134,2301.384155,2308.516683,2071.359098,1871.617981
